In [1]:
import os

import numpy as np
import tifffile
import scipy.ndimage as ndi
from matplotlib import pyplot as plt
from skimage import measure
import pandas as pd
import seaborn as sns
from nd2reader import ND2Reader

In [ ]:
df = pd.read_pickle("/path/to/output_data.pkl")
df.columns, len(df)

In [6]:
rowmap = {}
for i, a in enumerate('ABCDEFGHIJKLMNOPQRSTUVW'):
    rowmap[i + 1] = a

df['row_name'] = df.row.map(rowmap)

In [ ]:
df['well'] = df.row_name + df.col.astype('str').str.pad(2, fillchar='0')
df.sample(5)

In [8]:
df[(df.row==3) & (df.col==3)].well.unique()

array(['C03'], dtype=object)

In [ ]:
ex = pd.ExcelFile('/excel/plate/map/xlsx')
sheet = ex.sheet_names[1]
sheet0 = ex.sheet_names[2]
sheet

In [11]:
usecols = list(range(23))
map0_wide = pd.read_excel(ex, sheet_name=sheet0, index_col=0,
                          usecols=usecols, nrows=16).reset_index()
map0 = map0_wide.melt(['index'])
map0.columns=['row_name', 'col', 'mutant']


In [13]:
map0 = map0.dropna()

In [14]:
rowmap_reverse = {}
for i, a in enumerate('ABCDEFGHIJKLMNOPQRSTUVW'):
    rowmap_reverse[a] = int(i + 1)

map0['row'] = map0.row_name.map(rowmap_reverse).astype('int')

In [ ]:
map0['well'] = map0.row_name + map0.col.astype('str').str.pad(2, fillchar='0')
map0.sample(6)
    

In [ ]:
xmap = map0[['well', 'mutant']]
xmap.sample(4)

In [17]:
xmap.mutant.unique().shape

(85,)

In [18]:
df = df.merge(xmap, on=['well'])

In [19]:
## save to csv
df.to_csv("output_data.csv", float_format="%.3f")